In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from colormath.color_objects import sRGBColor, XYZColor
from colormath.color_conversions import convert_color
import scipy.interpolate as intp
from scipy.optimize import minimize
%matplotlib

Using matplotlib backend: GTK3Agg


In [2]:
MF = np.genfromtxt("CMF_5nm.csv", delimiter=',') 
print(MF.shape)
CMF = MF[:, 1:4]
ABeam = np.ones(CMF.shape[0])

(89, 4)


In [3]:
def clip2rgb(xyz):
    XYZ = XYZColor(xyz[0], xyz[1], xyz[2])
    crgb = convert_color(XYZ, sRGBColor)
    r = crgb.rgb_r
    g = crgb.rgb_g
    b = crgb.rgb_b
    r = r if r > 0 else 0
    g = g if g > 0 else 0
    b = b if b > 0 else 0
    r = r if r < 1 else 1
    g = g if g < 1 else 1
    b = b if b < 1 else 1
    return [r, g, b]

In [4]:
def XYZ2xyY(X, Y, Z):
    r = X + Y + Z
    x = X/r
    y = Y/r
    return [x, y, Y]

In [5]:
def g_euc(r1, r2):
    g = 0
    for i, _ in enumerate(r1):
        g+= (r1[i] - r2[i])**2
    return np.sqrt(g)

## [1] Fit Chromaticity Curve to a B-spline in parameterized xy coordinants

In [6]:
xvec = []
yvec = []
for c in CMF:
    [x, y, _] = XYZ2xyY(c[0], c[1], c[2])
    xvec.append(x)
    yvec.append(y)

s = np.linspace(0, 1, len(CMF)) # spline coordinant
spline = intp.make_interp_spline(s, np.c_[xvec, yvec])

def s2wl(s, spline, cmf):
    """Convert s-coordinant to a wavelength index
        by matching closest point in cmf
    """
    xyset = [XYZ2xyY(c[0], c[1], c[2])[0:2] for c in cmf]
    distset = [g_euc(spline(s), p) for p in xyset]
    return np.argmin(distset)

In [7]:
s = np.linspace(0, 1, 100)
plt.plot(spline(s)[:,0], spline(s)[:,1])

## [2] Define the white point and complementary function

In [8]:
wtXYZ = np.dot(ABeam, CMF)
wtxy = XYZ2xyY(wtXYZ[0], wtXYZ[1], wtXYZ[2])[0:2]

In [9]:
def line(x, m, b):
    """Function for a line"""
    return m*x+b

def g_line2locus(s, xy, wtpt, spline): 
    """distance between line xy-wtpt and locus"""
    slope = (wtpt[1] - xy[1])/(wtpt[0] - xy[0])
    b = wtpt[1] - slope*wtpt[0]
    if type(s) == np.ndarray: # correction for minimization use
        s = s[0]
    xy1 = [spline(s)[0], spline(s)[1]]
    xy2 = [spline(s)[0], line(spline(s)[0], slope, b)]
    return g_euc(xy1, xy2) 

def comp0(spline, wtpt):
    """Return complementary spline-coodinant of 0"""
    wlxy = np.array(spline(0))
    wtpt = np.array(wtpt)
    st = minimize(fun=g_line2locus, x0=0.7, bounds=[(0.1, 0.999)], args=(wlxy, wtpt, spline))
    return st

def comp1(spline, wtpt):
    """Return complementary spline-coodinant of 1"""
    wlxy = np.array(spline(1))
    wtpt = np.array(wtpt)
    st = minimize(fun=g_line2locus, x0=0.2, bounds=[(0, 0.3)], args=(wlxy, wtpt, spline))
    return st

def comp(s, spline, wtpt):
    """Return complementary spline-coodinant of s
    careful: if s is a green color, then there may be no complementary wl"""
    c0 = comp0(spline, wtpt).x[0]
    c1 = comp1(spline, wtpt).x[0]
    wlxy = np.array(spline(s))
    wtpt = np.array(wtpt)
    if 0 <= s and s < c1:
        # between lam0 and bar{lam}f
        return minimize(fun=g_line2locus, x0=0.7, bounds=[(0.1, 0.999)], args=(wlxy, wtpt, spline)).x[0]
    elif c1 <= s and s < c0:
        # between bar{lam}f and bar{lam}0
        return 1
    elif c0 <= s and s <=1:
        return minimize(fun=g_line2locus, x0=0.7, bounds=[(0.1, 0.999)], args=(wlxy, wtpt, spline)).x[1]

In [10]:
result0 = comp0(spline, wtxy)
result1 = comp1(spline, wtxy)
xy0 = spline(0)
[cxy0] = spline(result0.x)
xy1 = spline(1)
[cxy1] = spline(result1.x)
s = np.linspace(0, 1, 100)
plt.plot(spline(s)[:,0], spline(s)[:,1])
plt.plot((xy0[0], cxy0[0]), (xy0[1], cxy0[1]))
print(xy1, cxy1)
plt.plot((xy1[0], cxy1[0]), (xy1[1], cxy1[1]))
plt.scatter(wtxy[0], wtxy[1])

[0.71416994 0.28583006] [0.02795488 0.37135672]


In [11]:
print(comp0(spline, wtxy).x)

[0.40866544]


## [3] define semichrome map

In [12]:
def semi(s, spline, wtpt, cmf, sign=True):
    c0 = comp0(spline, wtpt).x
    c1 = comp1(spline, wtpt).x
    if 0 <= s and s < c1:
        # between lam0 and bar{lam}f
        c = comp(s, spline, wtpt)
    elif c1 <= s and s < c0:
        # between bar{lam}f and bar{lam}0
        c = 1
    elif c0 <= s and s <=1:
        c = comp(s, spline, wtpt)
        sign = not sign
    # now construct optimal functions
    L = len(cmf)
    piout = np.zeros((L))
    l_i = s2wl(s, spline, cmf)
    l_f = s2wl(c, spline, cmf)
    if (sign == True):
        piout[l_i:l_f] = 1
    elif (sign == False):
        piout[0:l_i] = 1
        piout[l_f::] = 1
    return piout

In [13]:
s = np.linspace(0, 1, 100)
plt.plot(spline(s)[:,0], spline(s)[:,1])
plt.scatter(wtxy[0], wtxy[1])
plt.plot((xy0[0], cxy0[0]), (xy0[1], cxy0[1]))

In [14]:
def bump(I, x0, xf):
    out = np.zeros(len(I))
    if x0 < xf:
        out[x0:xf] = 1
    elif x0 > xf:
        out[x0:] = 1
        out[0:xf] = 1
    return out

def Optimal(wl1, wl2, cmf, normalized=False):
    """
    Optimal windowing function with wrapping
    """
    X =  np.dot(cmf[:,0], bump(cmf, wl1, wl2))
    Y =  np.dot(cmf[:,1], bump(cmf, wl1, wl2))
    Z =  np.dot(cmf[:,2], bump(cmf, wl1, wl2))
    XYZ = [X, Y, Z]
    if normalized:
        Norm = sum(CMF)
        return (XYZ[0]/Norm[0], XYZ[1]/Norm[1], XYZ[2]/Norm[2])
    else:
        return (XYZ[0], XYZ[1], XYZ[2])

In [15]:
C0 = []
C1 = []
C2 = []
RGB = []
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_xlim(sum(CMF[:,0]))
ax.set_ylim(sum(CMF[:,1]))
ax.set_zlim(sum(CMF[:,2]))
L = CMF.shape[0]
for i in range(0, L, 1):
    for j in range(0, L, 1):
        O = Optimal(i, (i+j)%L, CMF)
        C0.append( O[0] )
        C1.append( O[1] )
        C2.append( O[2] )
        C = XYZColor(O[0]/sum(CMF)[0], O[1]/sum(CMF)[1], O[2]/sum(CMF)[2])
        crgb = convert_color(C, sRGBColor)
        r = crgb.rgb_r
        g = crgb.rgb_g
        b = crgb.rgb_b
        r = r if r > 0 else 0
        g = g if g > 0 else 0
        b = b if b > 0 else 0
        r = r if r < 1 else 1
        g = g if g < 1 else 1
        b = b if b < 1 else 1
        rgb = [r, g, b]
        RGB.append(rgb)
ax.scatter(C0, C1, C2, c=RGB)

c0 = comp0(spline, wtxy).x[0]
I = np.linspace(0, c0, 20)
for i in I:
    Pip = semi(i, spline, wtxy, CMF)
    cpi = np.dot(Pip, CMF)
    rgb = clip2rgb(cpi)
    ax.scatter(cpi[0], cpi[1], cpi[2], color=rgb, s=500)
    Pim = semi(i, spline, wtxy, CMF, False)
    cpi = np.dot(Pim, CMF)
    rgb = clip2rgb(cpi)
    ax.scatter(cpi[0], cpi[1], cpi[2], color=rgb, s=500)

IndexError: index 1 is out of bounds for axis 0 with size 1

## [4] find semichromes $\Pi^+_{\lambda, \bar{\lambda}}$ for $\lambda \in [\lambda_0, \bar{\lambda}_0]$